In [ ]:
%matplotlib notebook

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.stats as stats
from source.spectral import Spectral
np.random.seed(0)

# Initializing spectral model

In [ ]:
multiplier = 1
eps = 0.03253176201497791
eps *= multiplier
initialization = 'random' #'random', 'separated_x', 'separated_y', 'trivial_x', 'trivial_y', 
Lx = Ly = 1
Nx = 100
Ny = 100
dt, Nt = 5e-5, 1
show_error=False

spec = Spectral(Lx, Ly, Nx, Ny, eps, initialization)
spec.step = spec.step_method5

In [ ]:
plot = 0                  # 1: filled contour, 2: just level curves, else: greyscale image
# |k|^4 plot:
plt.xlabel("x interval")

if plot==1:
    plt.contourf(spec.y, spec.x, spec.k2**2, cmap=plt.cm.gray)
elif plot==2:
    plt.contour(spec.y, spec.x, spec.k2**2, cmap=plt.cm.gray)
else:
    plt.imshow(spec.k2**2, extent=(0,1,0,1), cmap=plt.cm.gray, origin='lower')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].set_title("Visualization")
axs[0].set_xlabel("X axis")
axs[0].set_ylabel("Y axis")
axs[1].set_title("Histogram of relative concentrations on grid points")
axs[1].set_xlabel("Relative concentration")

plot=1
bins=levels=32

if(plot==1):
    axs[0].contourf(spec.y, spec.x, spec.u, levels=levels, cmap=plt.cm.gray)
if(plot==2):
    axs[0].contour(spec.y, spec.x, spec.u, levels=levels, cmap=plt.cm.gray)
else:
    image = axs[0].imshow(spec.u, cmap=plt.cm.gray, origin='lower', animated=True)

if show_error:
    error_info = axs[0].text(0.5,0.85, f"Difference: {0}",
        bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
        transform=axs[0].transAxes, ha="center")

histogram = axs[1].hist(spec.u.flatten(), bins=bins)

def animate(i):
    spec.advance(dt, Nt)
    if(plot==1):
        axs[0].clear()
        axs[0].set_xlabel("X axis")
        axs[0].set_ylabel("Y axis")
        axs[0].contourf(spec.y, spec.x, spec.u, levels=levels, cmap=plt.cm.gray)
    if(plot==2):
        axs[0].clear()
        axs[0].contour(spec.y, spec.x, spec.u, levels=levels, cmap=plt.cm.gray)
    else:
        image.set_data(spec.u)
    axs[1].cla()
    distribution = spec.u.flatten()

    histogram = axs[1].hist(distribution, bins=bins)
    if show_error:
#         err, = [np.log(np.abs(np.sum(spec.u)/(Nx*Ny)/deviation - 1))/np.log(10)
        err, = [(np.abs(np.sum(spec.u)/(Nx*Ny) - deviation))
        if deviation!=0 else -np.inf]
        error_info.set_text(f"Log-error: {err}")
    return 

In [ ]:
anim = animation.FuncAnimation(fig=fig, func=animate)

fig.show()

In [ ]:
"""Press Ctrl+Return on this cell to monitor the concentrations"""
temp = f"Total: {spec.u.size}", f"Below viable: {spec.u[spec.u>1].size}", f"Above viable: {spec.u[spec.u<-1].size}"
print(temp)